In [4]:
! pip install -q --upgrade langchain langchain-openai langchain-core langchain_community docx2txt pypdf  langchain_chroma sentence_transformers

In [5]:
import langchain
print(langchain.__version__)

0.3.9


In [7]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langchain-course"

In [11]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")
llm_response = llm.invoke("Tell me a joke")

llm_response


AIMessage(content="Why don't scientists trust atoms? \n\nBecause they make up everything!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 11, 'total_tokens': 24, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0705bf87c0', 'finish_reason': 'stop', 'logprobs': None}, id='run-c439a559-3894-48e8-b8c7-54422a6819dd-0', usage_metadata={'input_tokens': 11, 'output_tokens': 13, 'total_tokens': 24, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [9]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
output_parser.invoke(llm_response)

"Why don't scientists trust atoms?\n\nBecause they make up everything!"

In [10]:
chain = llm | output_parser
chain.invoke("Tell me a joke")

'Why did the scarecrow win an award? \n\nBecause he was outstanding in his field!'

In [12]:
from typing import List
from pydantic import BaseModel, Field

class MobileReview(BaseModel):
    phone_model: str = Field(description="Name and model of the phone")
    rating: float = Field(description="Overall rating out of 5")
    pros: List[str] = Field(description="List of positive aspects")
    cons: List[str] = Field(description="List of negative aspects")
    summary: str = Field(description="Brief summary of the review")

review_text = """
Just got my hands on the new Galaxy S21 and wow, this thing is slick! The screen is gorgeous,
colors pop like crazy. Camera's insane too, especially at night - my Insta game's never been
stronger. Battery life's solid, lasts me all day no problem.

Not gonna lie though, it's pretty pricey. And what's with ditching the charger? C'mon Samsung.
Also, still getting used to the new button layout, keep hitting Bixby by mistake.

Overall, I'd say it's a solid 4 out of 5. Great phone, but a few annoying quirks keep it from
being perfect. If you're due for an upgrade, definitely worth checking out!
"""

structured_llm = llm.with_structured_output(MobileReview)
output = structured_llm.invoke(review_text)
output

MobileReview(phone_model='Galaxy S21', rating=4.0, pros=['Gorgeous screen', 'Vibrant colors', 'Insane camera performance, especially at night', 'Solid battery life'], cons=['Pretty pricey', 'No charger included', 'New button layout takes time to get used to'], summary='Overall, a solid phone with great features, but a few annoying quirks prevent it from being perfect.')

In [14]:
output.summary

'Overall, a solid phone with great features, but a few annoying quirks prevent it from being perfect.'

In [17]:
from langchain_core.prompts import ChatPromptTemplate
prompt= ChatPromptTemplate.from_template("tell me joke about {topic}")
prompt.invoke({"topic":"programmer"})

ChatPromptValue(messages=[HumanMessage(content='tell me joke about programmer', additional_kwargs={}, response_metadata={})])

In [22]:
chain =prompt | llm | output_parser
chain.invoke({"topic":"Sales People"})

'Why did the sales person bring a ladder to work?\n\nBecause they heard the job was all about reaching new heights!'

In [23]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, SystemMessage

system_message = SystemMessage(content="You are a helpful assistant that tells jokes.")
human_message = HumanMessage(content="Tell me about programming")
llm.invoke([system_message, human_message])

AIMessage(content='Sure! Here’s a light-hearted joke about programming:\n\nWhy do programmers prefer dark mode?\n\nBecause light attracts bugs! \n\nIf you have more specific questions about programming, feel free to ask!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 24, 'total_tokens': 62, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0705bf87c0', 'finish_reason': 'stop', 'logprobs': None}, id='run-16f6c8b2-fd92-4500-82c8-158cdf2d1915-0', usage_metadata={'input_tokens': 24, 'output_tokens': 38, 'total_tokens': 62, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})